In [1]:
# 문제 1: 카페 메뉴 도구 호출 체인 구현

import re
import os
from textwrap import dedent
from pprint import pprint
from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

In [2]:

# 환경 설정
load_dotenv()


True

In [3]:

# 1. 카페 메뉴 데이터 로드 및 벡터 DB 구축
def create_cafe_vector_db():
    # 카페 메뉴 텍스트 데이터를 로드
    loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
    documents = loader.load()
    
    # 메뉴 항목별로 분할
    def split_menu_items(document):
        pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
        menu_items = re.findall(pattern, document.page_content, re.DOTALL)
        
        menu_documents = []
        for i, item in enumerate(menu_items, 1):
            # 메뉴 이름 추출
            menu_name = item.split('\n')[0].split('.', 1)[1].strip()
            
            menu_doc = Document(
                page_content=item.strip(),
                metadata={
                    "source": document.metadata['source'],
                    "menu_number": i,
                    "menu_name": menu_name
                }
            )
            menu_documents.append(menu_doc)
        
        return menu_documents
    
    # 메뉴 항목 분리 실행
    menu_documents = []
    for doc in documents:
        menu_documents += split_menu_items(doc)
    
    # 임베딩 모델 설정
    embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
    
    # FAISS 인덱스 생성
    cafe_db = FAISS.from_documents(
        documents=menu_documents, 
        embedding=embeddings_model
    )
    
    # FAISS 인덱스 저장
    cafe_db.save_local("./db/cafe_db")
    
    return cafe_db


In [4]:

# 2. 도구 정의
# 웹 검색 도구
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])
    
    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."


In [5]:

# 위키피디아 검색 도구
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field

def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()
    
    formatted_docs = [
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
    ]
    
    return formatted_docs

class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)


In [6]:

# LLM 모델 
#llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm.model_name)

meta-llama/llama-4-scout-17b-16e-instruct


In [7]:

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm
)

wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)


C:\Users\user\AppData\Local\Temp\ipykernel_2228\4223853708.py:6: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_summary = summary_chain.as_tool(


In [8]:

# 카페 메뉴 검색 도구
@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for cafe menu-related queries to maintain data confidentiality.
    """
    embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
    cafe_db = FAISS.load_local(
        "./db/cafe_db", 
        embeddings_model, 
        allow_dangerous_deserialization=True
    )
    
    docs = cafe_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 카페 메뉴 정보를 찾을 수 없습니다.")]


In [9]:

# 3. 도구를 LLM에 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)


In [10]:

# 4. 간단한 도구 호출 체인 구현
@chain
def cafe_search_chain(user_input: str, config: RunnableConfig):
    # 첫 번째 LLM 호출로 도구 사용 결정
    ai_msg = llm_with_tools.invoke(user_input, config=config)
    
    # 도구 실행
    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
    
    # 최종 답변 생성을 위한 프롬프트
    final_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful cafe assistant. Provide accurate information based on the search results."),
        ("human", "{user_input}"),
        ("ai", ai_msg.content if ai_msg.content else "도구를 사용하여 정보를 검색했습니다."),
        ("human", "검색 결과: {tool_results}")
    ])
    
    # 도구 결과를 문자열로 변환
    tool_results_str = "\n\n".join([str(msg.content) for msg in tool_msgs])
    
    # 최종 답변 생성
    final_chain = final_prompt | llm
    return final_chain.invoke({
        "user_input": user_input,
        "tool_results": tool_results_str
    }, config=config)


In [12]:

# 5. 실행 및 테스트
if __name__ == "__main__":
    # 벡터 DB 생성 (최초 1회만 실행)
    try:
        create_cafe_vector_db()
        print("카페 메뉴 벡터 DB가 성공적으로 생성되었습니다.")
    except Exception as e:
        print(f"벡터 DB 생성 중 오류: {e}")
    
    # 질문에 답변
    query = "아메리카노의 가격은 얼마인가요? 아메리카노의 유래는 무엇인가?"
    response = cafe_search_chain.invoke(query)
    
    print("질문:", query)
    print("답변:", response.content)                                      

카페 메뉴 벡터 DB가 성공적으로 생성되었습니다.
db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '아메리오노 가격'}, 'id': '5ev3c1342', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': 'Americano 유래'}, 'id': 'jcnv2y5v8', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
질문: 아메리카노의 가격은 얼마인가요? 아메리카노의 유래는 무엇인가?
답변: 아메리카노의 가격은 4,500원입니다.

아메리카노의 유래는 1940년대 이탈리아에 있습니다. 2차 세계 대전 당시 이탈리아에 주둔하던 미군이 에스프레소에 뜨거운 물을 넣어 마셨는데, 이것이 아메리카노의 시작입니다.
